In [ ]:
# Requirements

# !pip install llama-index
# !pip install llama-index-core
# !pip install llama-index-embeddings-openai
# !pip install llama-parse
# !pip install llama-index-vector-stores-kdbai
# !pip install pandas
# !pip install llama-index-postprocessor-cohere-rerank
# !pip install kdbai_client

In [ ]:
# Import Required Libraries

from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.vector_stores.kdbai import KDBAIVectorStore
from llama_index.embeddings.openai import OpenAIEmbedding   
from llama_index.core import VectorStoreIndex  
from llama_index.core import StorageContext
from llama_index.llms.openai import OpenAI     
from llama_index.core import Settings
from llama_parse import LlamaParse
import kdbai_client as kdbai
import pickle
import os

In [ ]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio

import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

KDBAI_ENDPOINT = (
    os.environ["KDBAI_ENDPOINT"]
)
KDBAI_API_KEY = (
    os.environ["KDBAI_API_KEY"]
)

#connect to KDB.AI
session = kdbai.Session(api_key=KDBAI_API_KEY, endpoint=KDBAI_ENDPOINT)

In [ ]:
# The schema contains two metadata columns (document_id, text) and one embeddings column
# Index type, search metric (Euclidean distance), and dimensions are specified in the embedding column

schema = dict(
    columns=[
        dict(name="document_id", pytype="bytes"),
        dict(name="text", pytype="bytes"),
        dict(
            name="embedding",
            vectorIndex=dict(type="flat", metric="L2", dims=1536),
        ),
    ]
)

KDBAI_TABLE_NAME = "LlamaParse_Table"

# First ensure the table does not already exist
if KDBAI_TABLE_NAME in session.list():
    session.table(KDBAI_TABLE_NAME).drop()

#Create the table
table = session.create_table(KDBAI_TABLE_NAME, schema)

In [ ]:
EMBEDDING_MODEL  = "text-embedding-3-small"
GENERATION_MODEL = "gpt-4o"

llm = OpenAI(model=GENERATION_MODEL)
embed_model = OpenAIEmbedding(model=EMBEDDING_MODEL)

Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
# Create Custom Parsing Instructions

parsing_instructions = '''Answer questions using the information in this pdf and be precise. Avoid Hallucinations, and say you don't know if given data is not enough to answer the question'''

In [ ]:
name = "Resume"
path = os.path.join('.', 'data', f'{name}.pdf')
pickle_path = os.path.join('.', 'data', f'parsed_{name}_documents.pkl')

if os.path.exists(pickle_path):
    with open(pickle_path, 'rb') as file:
        documents= pickle.load(file)
        print("Loaded documents")
else:
    documents = LlamaParse(result_type="markdown", parsing_instructions=parsing_instructions).load_data(path)
    with open(pickle_path, 'wb') as pickle_file:
        pickle.dump(documents, pickle_file)

In [ ]:
# Parse the documents using MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(llm=llm, num_workers=8).from_defaults()

# Retrieve nodes (text) and objects (table)
nodes = node_parser.get_nodes_from_documents(documents)

base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

In [ ]:
vector_store = KDBAIVectorStore(table)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

#Create the index, inserts base_nodes and objects into KDB.AI
recursive_index = VectorStoreIndex(
    nodes= base_nodes + objects, storage_context=storage_context
)

# Query KDB.AI to ensure the nodes were inserted
table.query()

In [ ]:
### Define reranker
cohere_rerank = CohereRerank(top_n=10)

### Create the query_engine to execute RAG pipeline using LlamaIndex, KDB.AI, and Cohere reranker
query_engine = recursive_index.as_query_engine(similarity_top_k=15, node_postprocessors=[cohere_rerank])

In [ ]:
query_1 = '''Who is Aditya Kushal'''

response_1 = query_engine.query(query_1)

print(str(response_1))